#### CAS AI Chat Bot 시스템 개발

1. pip install list
### transformers==4.18.0 
### torch==1.10.0

In [4]:
%pip install transformers -q
%pip install torch -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install -q -U git+https://github.com/huggingface/peft.git -q
%pip install -q -U git+https://github.com/huggingface/accelerate.git -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

In [2]:
model_path = "/data/bwllm/models/KoAlpaca-Polyglot-12.8B"
model = AutoModelForCausalLM.from_pretrained(model_path)          
tokenizer = AutoTokenizer.from_pretrained(model_path)

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [3]:
print(model)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30003, 5120)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-39): 40 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=5120, out_features=15360, bias=True)
          (dense): Linear(in_features=5120, out_features=5120, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=5120, out_features=20480, bias=True)
          (dense_4h_to_h): Linear(in_features=20480, out_features=5120, bias=True)

In [4]:
print(tokenizer)

PreTrainedTokenizerFast(name_or_path='/data/bwllm/models/KoAlpaca-Polyglot-12.8B', vocab_size=30000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|endoftext|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|endoftext|>', '<|sep|>', '<|acc|>', '<|tel|>', '<|rrn|>']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<|unused0|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<|unused1|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<|sep|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	30000: AddedToken("<|acc|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	30001: Add

In [5]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))
    tokenizer.pad_token = tokenizer.eos_token
    
tokenizer.pad_token = tokenizer.eos_token    

In [6]:
query = "대한민국 수도는 어디지"
inputs = tokenizer.encode_plus(query, return_tensors="pt")
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
input_ids

tensor([[7635, 2248,  272, 2116,  295]])

In [7]:
generate_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=512)
generate_ids

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [23]:
generated_text = tokenizer.decode(generate_ids[0], skip_special_tokens=True)
generated_text

TypeError: argument 'ids': 'list' object cannot be interpreted as an integer